# IMPORTS

In [1]:
import numpy as np 
import tqdm
from numpy import ndarray as nd 
from glob import glob 
from os.path import join
from pymatreader import read_mat
from matplotlib import pyplot as plt 
from utils import *
import torch
%matplotlib inline

# training_data_location = "D:\BayesianECGI\Auckland\Pig1\Signals"

In [2]:
GEOM = read_mat("D:\\BayesianECGI\\Utilities\\Geometries\\epigeom490corrected.mat")
PTS = GEOM["epigeom490corrected"]["pts"]
FAC = GEOM["epigeom490corrected"]["fac"]
A = read_mat("D:\\BayesianECGI\\Utilities\\Geometries\\ForwMat_HLT.mat")
A = A["Trf_HLT_leads"]

# TRAINING DATA

In [ ]:
training_data_location = join("D:\BayesianECGI\Bayesian\TrainingData","features")
files = glob(join(training_data_location,"*.mat"))
len(files)

In [ ]:
X = np.empty((490,1))
Y = np.empty((192,1))
AT = np.empty((490,1))
intervals = []

In [ ]:
for i,file in enumerate(files): 
    data = read_mat(file)
    begin = data["features"]["QRSbegin"]
    end = data["features"]["QRSend"]
    interval = end-begin 
    intervals.append(interval)
    QRS = data["ts"]["potvals"][:,begin:end]
    if np.sum(np.logical_or(QRS<-60,QRS>60)):
        continue
    at = data["features"]["AT"] - begin
    # assert np.max(at)<=interval, (print(np.max(at), interval) and False) or "AT out of QRS!"
    X = np.hstack((X,QRS))
    AT = np.hstack((AT,np.expand_dims(at,1)))
    
X = X[:,1:]
AT = AT[:,1:]


In [ ]:
plt.figure(figsize=(24,6))
plt.plot(X.T);

In [ ]:
plt.figure(figsize=(24,6))
plt.plot(X[:,5000:15000].T);

In [ ]:
plt.figure(figsize=(24,6))
plt.plot(X[:,0:2000].T);
plt.grid()

# There exists 27951 samples, For convenience for the temporal batch_size dependency, I am going to slice it to 8192x3+1024x2 =  26624, and use the validation ratio in two's powers

In [ ]:
X = X[:,0:26624]
X.shape

In [ ]:
Y = A.dot(X)
Y.shape

In [ ]:
plt.subplot(2,1,1)
plt.imshow(Y[:,:100],extent=[0,1,0,1]);
plt.subplot(2,1,2)
plt.imshow(X[:,:100],extent=[0,1,0,1]);

In [ ]:
ylim = [-30,30]
plt.figure(figsize=(24,18))
plt.subplot(3,1,1)
plt.plot(Y.T);
plt.grid()
plt.ylim(ylim)
Y,N,_ = add_noise(Y,20)
plt.subplot(3,1,2)
plt.plot(N.T);
plt.ylim(ylim)
plt.subplot(3,1,3)
plt.plot(Y.T);
plt.ylim(ylim)

In [ ]:
node_ordering = read_mat('newnode_order_3.mat')['node_order'] - 1 # For MATLAB to Python indexing, there is -1
node_ordering

In [ ]:
np.save("heart_concat_new.npy",X,allow_pickle=True)
np.save("torso_concat_new.npy",Y,allow_pickle=True)
np.save("AT_new.npy",AT,allow_pickle=True)
np.save("heart_concat_reordered_new.npy",X[node_ordering,:],allow_pickle=True)
np.save("torso_concat_reordered_new.npy",Y,allow_pickle=True)
np.save("AT_reordered_new.npy",AT[node_ordering,:],allow_pickle=True)
np.save("A_HLT_reordered.npy",A[:,node_ordering],allow_pickle=True)

# TEST DATA 

In [3]:
X_test = np.empty((490,1))
Y_test = np.empty((192,1))
AT_test = np.empty((490,1))
intervals = []
test_data_location = join("D:\BayesianECGI\Bayesian\TestData","EP")
test_files = glob(join(test_data_location,"*.mat"))
test_files

['D:\\BayesianECGI\\Bayesian\\TestData\\EP\\qrs_21jun01_12.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\qrs_21jun01_3.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\qrs_21jun01_4.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\qrs_8oct02_31.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\qrs_8oct02_32.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm131200_13qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0055_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0056_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0066_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0082_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0086_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0090_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0120_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0123_qrs.mat',
 'D:\\BayesianECGI\\Bayesian\\TestData\\EP\\rsm8oct02_0130_qr

In [ ]:
test_data_container = []
test_data_counter = np.array([])
for i,file in enumerate(test_files):
    data_container = {} 
    data = read_mat(file)
    QRS = data["ep"]["potvals"]
    time_frames = QRS.shape[-1]
    test_data_counter = np.hstack((test_data_counter,(np.ones((time_frames,1))*(i)).flatten()))
    at = data["ep"]["at"]
    noise_free_torso = A.dot(QRS)
    noisy_torso, N, std_noise = add_noise(noise_free_torso,20)
    # assert np.max(at)<=interval, (print(np.max(at), interval) and False) or "AT out of QRS!"
    X_test = np.hstack((X_test,QRS))
    AT_test = np.hstack((AT_test,np.expand_dims(at,1)))
    data_container['x'] = torch.from_numpy(QRS[node_ordering,:])
    data_container['y'] = torch.from_numpy(noisy_torso)
    data_container['badleads'] = torch.from_numpy(np.where(np.in1d(node_ordering,data['ep']['badleads']-1))[0]) # -1 Due to MATLAB ordering
    data_container['std_n'] = std_noise
    data_container['at'] = at[node_ordering]
    data_container['paceloc'] = torch.from_numpy(np.where(np.in1d(node_ordering,data['ep']['pacing']-1))[0]) 
    test_data_container.append(data_container)    
X_test = X_test[:,1:]
AT_test = AT_test[:,1:]

In [ ]:
Y_test = A.dot(X_test)

In [ ]:
[item['x'].shape[1] for item in test_data_container]

In [ ]:
ylim = [-15,15]
plt.figure(figsize=(24,18))
plt.subplot(4,1,1)
plt.plot(X_test.T);
plt.grid()
Y_test_noisy,N,_ = add_noise(Y_test,20)
plt.subplot(4,1,2)
plt.plot(N.T);
plt.ylim(ylim)
plt.subplot(4,1,3)
plt.plot(Y_test.T);
plt.ylim(ylim)
plt.subplot(4,1,4)
plt.plot(Y_test_noisy.T);
plt.ylim(ylim)

In [ ]:
np.save("test_heart_concat.npy",X_test,allow_pickle=True)
np.save("test_torso_concat.npy",Y_test,allow_pickle=True)
np.save("test_AT.npy",AT_test,allow_pickle=True)
np.save("test_heart_concat_reordered.npy",X_test[node_ordering,:],allow_pickle=True)
np.save("test_torso_concat_reordered.npy",Y_test,allow_pickle=True)
np.save("test_AT_reordered.npy",AT_test[node_ordering,:],allow_pickle=True)
np.save("test_data_counter.npy",test_data_counter,allow_pickle=True)


In [ ]:
torch.save(test_data_container,'test_dictionary.pt')

# Visualization 

In [ ]:
from models import DFBlock
import torch
x_hat = DFBlock(torch.from_numpy(A), torch.Tensor([1.75e-8]), torch.from_numpy(Y), torch.zeros_like(torch.from_numpy(X)), 'cpu', 1)
                   

In [ ]:
plt.subplot(2,1,1)
plt.imshow(X[:,:100],extent=[0,1,0,1]);
plt.title('Ground-truth')
plt.subplot(2,1,2)
plt.imshow(x_hat[:,:100].numpy(),extent=[0,1,0,1]);
plt.title('Tikhonov solution')
plt.tight_layout()